In [49]:
import os
import json
import re

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
# from azure.search.documents.aio import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [50]:
# user_input = "¿Qué dice el artículo 103 del Código Penal?"  # 1
# user_input = "¿Cuáles son las causales de agravación del homicidio?"  # 2
# user_input = "Si A le roba el celular a B, ¿qué delito comete?"  # 3
# user_input = "¿Cuál es la diferencia entre dolo eventual y culpa con representación?"  # 4
# user_input = "¿Cuáles son las formas de autoría en derecho penal?"  # 5
# user_input = "¿Cuándo se consuma el hurto?"  # 6
# user_input = "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?"  # 7
# user_input = "¿Cuál es el término que tiene la Fiscalía para formular acusación?"  # 8
# user_input = "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"  # 9
# user_input = "¿Se puede precluir una investigación antes de formular imputación?"  # 10
# user_input = "¿Cuáles son las causales de ausencia de responsabilidad en el derecho penal?"  # 11
# user_input = "¿Cuáles son las causales genéricas de atenuación punitiva?"  # 12
# user_input = "¿Qué se entiende por “documento” en el derecho penal?"  # 13
# user_input = "¿Cuáles son las causales de agravación de la estafa?" #14
# user_input = "¿Se pueden realizar preguntas sugestivas en el contrainterrogatorio del juicio oral?" #15
# user_input = "¿Un juez debe ordenar la captura del procesado cuando anuncie sentido de fallo condenatorio?" #16
# user_input = "¿Cuáles son las fuentes de posición de garante según el código penal?" #17
# user_input = "¿La búsqueda selectiva en base de datos requiere control previo?" #18
# user_input = "¿Cuándo procede la captura en flagrancia?" #19
# user_input = "Si alguien falsifica un documento y lo usa como medio para engañar a otra persona y llevarlo a que le pague algo que no le debe, ¿qué delito(s) comete?" #20

# user_input = "Cuál es la sanción por no presentar la declaración de renta a timepo?"
# user_input = "Qué es el derecho a la vida?"
# user_input = "En qué artículo de la constitución está consagrado el derecho a la vida?"
# user_input = "En qué artículo del código penal se contempla la definición de documento?"
# user_input = "¿Qué dice el artículo 14 del código penal?"
user_input = "¿Qué dice el artículo 14 de la ley 599 del 2000?"
# user_input = "¿Qué dice el artículo 241 del código penal?"
# user_input = "¿Qué se entiende por “documento” en el código penal?"
# user_input = "¿Qué dice el artículo 249 del código penal?"
# user_input = "¿Qué dice el artículo 221 del código penal?"
# user_input = "¿En qué artículo se encuentra el delito de calumnia?"
# user_input = "Si B imputa falsamente a A un delito que no ha cometido ¿En qué conducta punible incurre?"
# user_input = "En qué delito incurre una persona que entre sin autorización en un sistema informático?"
# user_input = "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# user_input = "¿Qué delito comtió una persona que adquirió un bien cuyo origen inmediato es una actividad ilícita?"
# user_input = "¿Qué dice el artículo 323 del código penal?"
# user_input = "¿Qué artículo contiene el delito de lavado de activos en el código penal?"
# user_input = "¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?"
# user_input = "¿Dónde se considera realizada la conducta punible?"
# user_input = "Si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "Según el artículo 14 del código penal si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "¿Qué se entiende por servido público en código penal?"
# user_input = "¿Cuáles son las formas de autoría para el código penal?"

print("🧑 User input:")
print(user_input)

🧑 User input:
¿Qué dice el artículo 14 de la ley 599 del 2000?


In [51]:
query_system_template = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.
    You have access to Azure AI Search index with 100's of documents.
    Generate a search query based on the conversation and the new question.
    Do not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.
    Do not include any text inside [] or <<>> in the search query terms.
    Do not include any special characters like '+'.
    If the question is not in English, translate the question to English before generating the search query.
    If you cannot generate a search query, return just the number 0.
    """

query_few_shots = [
    {"role": "user", "content": "¿Qué dice el artículo 103 del Código Penal?"},
    {"role": "assistant", "content": '"Artículo 103" Ley 599 de 2000'},
    {
        "role": "user",
        "content": "¿Cuáles son las causales de agravación del homicidio?",
    },
    {
        "role": "assistant",
        "content": '"Causales de agravación del homicidio" "Ley 599 de 2000"',
    },
    {"role": "user", "content": "Si A le roba el celular a B, ¿qué delito comete?"},
    {"role": "assistant", "content": '"Hurto" "Ley 599 de 2000" "Celular"'},
    {
        "role": "user",
        "content": "¿En qué artículo de la Constitución Política se consagra el principio de dignidad humana?",
    },
    {
        "role": "assistant",
        "content": '"Principio de dignidad humana" "Constitución Política de Colombia"',
    },
    {"role": "user", "content": "¿Cuáles son las formas de autoría en derecho penal?"},
    {
        "role": "assistant",
        "content": 'Formas de autoría en derecho penal "Ley 599 de 2000"',
    },
    {"role": "user", "content": "¿Cuándo se consuma el hurto?"},
    {"role": "assistant", "content": '"Hurtoconsumación" "Ley 599 de 2000"'},
    {
        "role": "assistant",
        "content": '"Principio de dignidad humana" "Constitución Política de Colombia"',
    },
    {
        "role": "user",
        "content": "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?",
    },
    {
        "role": "assistant",
        "content": '"Personas jurídicas" "sujetos pasivos" "delitos de injuria y calumnia"',
    },
    {
        "role": "user",
        "content": "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?",
    },
    {
        "role": "assistant",
        "content": '"Archivo de investigación" "legítima defensa" "Ley 906 de 2004"',
    },
    {"role": "user", "content": "¿Cuáles son los requisitos de la denuncia?"},
    {"role": "assistant", "content": '"Requisitos de la denuncia" Ley 906 de 2004'},
]

query_user_content_template = f"Generate search query for: {user_input}."

query_messages = [
    {"role": "system", "content": query_system_template},
    *query_few_shots,
    {"role": "user", "content": query_user_content_template},
]

In [52]:
query_completion = (
    openai_client.chat.completions.create(
        messages=query_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.0,
        n=1,
    )
    .choices[0]
    .message.content
)

query = re.sub(r'"', "", query_completion)
# query = "servido público en código penal"

query_vectors = embeddings_client.generate_embeddings(content=query)

print("🔎 Query for:")
print(query)

🔎 Query for:
Artículo 14 Ley 599 de 2000


In [53]:
results = search_client.search(
    search_text=query,
    top=5,
    query_type=QueryType.SEMANTIC,
    vector_queries=[
        {
            "vector": query_vectors,
            "k": 5,
            "fields": "content_vector",
            "kind": "vector",
            "exhaustive": True,
        }
    ],
    # semantic_configuration_name="default",
    semantic_configuration_name="none",
    # semantic_query="",
    query_caption="extractive|highlight-false",
    scoring_profile="none",
    search_fields=["content"],
    # scoring_profile="legal",
    # scoring_parameters=[
    #     "tagx6-Constitucional",
    #     "tagx5-Legal",
    #     "tagx4-Infralegal",
    #     "tagx3-Jurisprudencia",
    #     "tagx2-Doctrina",
    # ],
)


def add_to_context(results, token_limit=3500):
    printer_documents = []

    results_list = list(results)

    top_results = results_list[:5]
    supporting_results = results_list[5:]

    documents = []

    for document in top_results:
        # print(document)
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "page": document["page"],
            "title": document["title"],
            "content": document["content"],
            "external_id": document["external_id"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }
        documents.append(doc_formatted)
        printer_documents.append(json.dumps(doc_formatted, indent=4))

    current_token_count = num_tokens_from_string(str(documents))

    additional_documents = []
    for document in supporting_results:
        captions: QueryCaptionResult = document["@search.captions"]
        captions_text = " // ".join([caption.text for caption in captions])
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "title": document["title"],
            "page": document["page"],
            "captions": captions_text,
            "external_id": document["external_id"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }

        new_token_count = current_token_count + num_tokens_from_string(
            str(doc_formatted)
        )

        if new_token_count <= token_limit:
            additional_documents.append(doc_formatted)
            printer_documents.append(json.dumps(doc_formatted, indent=4))
            current_token_count = new_token_count
        else:
            break

    response = {
        "sources": documents + additional_documents,
        "documents": documents,
        "printer_documents": printer_documents,
        "additional_documents": additional_documents,
        "token_count": current_token_count,
    }

    return response


sources = add_to_context(results)
documents = sources["sources"]

print("Legal profile - Query for: ")
print(query)
print("Num of docs:", len(documents), " ---  Tokens: ", sources["token_count"])
print("=============================")
print("\n".join(sources["printer_documents"]))

Legal profile - Query for: 
Artículo 14 Ley 599 de 2000
Num of docs: 5  ---  Tokens:  2505
{
    "id": "20240719190518externadoleccionesdederechopenalpartegeneralpdf_chunk203",
    "page": 145,
    "title": "EXTERNADO - Lecciones de Derecho Penal. Parte General",
    "content": "adoptada por la legislaci\u00f3n nacional, donde cobra relevancia tanto el lugar\ndonde se desarroll\u00f3 total o parcialmente la acci\u00f3n como donde se produjo o\ndebi\u00f3 producirse el resultado.\nPara el efecto, el art\u00edculo 14 de la Ley 599 de 2000, como anteriormente\nlo hac\u00eda el art\u00edculo 13 del Decreto 100 de 1980, fija unos par\u00e1metros claros,\nseg\u00fan los cuales la conducta punible se considera realizada:\na. En el lugar donde se desarroll\u00f3 total o parcialmente la acci\u00f3n (aspecto\nque resulta relevante en materia de extradici\u00f3n).\nb. En el sitio donde debi\u00f3 realizarse la acci\u00f3n omitida, en los casos de\ncomisi\u00f3n por omisi\u00f3n.\nc. Donde se prod

In [54]:
response_system_template = f'Eres Ariel, un asistente para la investigación legal \n\n    Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n\nCada fuente tiene un nombre seguido por dos puntos y la información real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. Asegúrate de capturar todo antes de ".pdf", por ejemplo: [SFC - Títulos valores electrónicos. Pagarés. Depósito centralizado de valores. Exigencia Concepto 2020086426-003 del 24 de junio de 2020 id2020086426.pdf]. No dividas los nombres de archivo por ninguna razón, ya que todos deben terminar en .pdf. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'

response_few_shots = []

response_user_content_template = f"{user_input} Sources: {documents}"

response_messages = [
    {"role": "system", "content": response_system_template},
    *query_few_shots,
    {"role": "user", "content": response_user_content_template},
]

In [55]:
response = (
    openai_client.chat.completions.create(
        messages=response_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.2,
        n=1,
    )
    .choices[0]
    .message.content
)

In [56]:
print(user_input)
print("Query: ", query)
print("")
print("=============================================")
print("")
print(response)
# dónde se considera realizada la conducta punible Ley 599 de 2000
# lugar de realización de la conducta punible Ley 599 de 2000
# lugar de realización de la conducta punible Ley 599 de 2000

# homicidio lugar de realización disparo en un lugar muerte en otro Ley 599 de 2000
# homicidio lugar de realización disparo en una ciudad muerte en otra

¿Qué dice el artículo 14 de la ley 599 del 2000?
Query:  Artículo 14 Ley 599 de 2000


<strong>Artículo 14 de la Ley 599 de 2000</strong>

El artículo 14 de la Ley 599 de 2000 establece el principio de territorialidad de la ley penal en Colombia. Según este artículo, la conducta punible se considera realizada en los siguientes lugares:

a. En el lugar donde se desarrolló total o parcialmente la acción.
b. En el sitio donde debió realizarse la acción omitida, en los casos de comisión por omisión.
c. Donde se produjo o debió producirse el resultado, para los eventos de los delitos tentados.

Estas reglas son particularmente útiles para asumir la investigación de delitos continuados y de ejecución permanente, aplicando la ley penal nacional cuando la acción se efectuó en Colombia, la acción omitida debió ejecutarse en Colombia, o los resultados de la omisión se producen en Colombia, así como en los delitos tentados cuando el resultado debió producirse en Colombia o allí tuvo lugar una par